In [1]:

from utils.path_utils import project_root

import os

import torch

import numpy as np
import pandas as pd

import tqdm


In [37]:
from utils.pretrain_utils.data import get_pretrain_finetune_test_datasets

pt_train, finetune, test = get_pretrain_finetune_test_datasets()

Test Set: 100%|██████████| 16000/16000 [00:04<00:00, 3932.98it/s]


Pre-training samples:  torch.Size([20336, 336, 40])
Fine-tuning samples:  torch.Size([4000, 336, 40]) Test samples:  torch.Size([16000, 336, 40])


In [29]:
!sbatch eval_simmtm.job

Submitted batch job 24928278


In [3]:
-

# Args

In [ ]:
import argparse

parser = argparse.ArgumentParser()

home_dir = os.getcwd()
parser.add_argument('--run_description', default='run1', type=str, help='Experiment Description')
parser.add_argument('--seed', default=2023, type=int, help='seed value')

parser.add_argument('--training_mode', default='pre_train', type=str, help='pre_train, fine_tune')
parser.add_argument('--pretrain_dataset', default='SleepEEG', type=str,
                    help='Dataset of choice: SleepEEG, FD_A, HAR, ECG')
parser.add_argument('--target_dataset', default='Epilepsy', type=str,
                    help='Dataset of choice: Epilepsy, FD_B, Gesture, EMG')

parser.add_argument('--logs_save_dir', default='experiments_logs', type=str, help='saving directory')
parser.add_argument('--device', default='cuda', type=str, help='cpu or cuda')
parser.add_argument('--home_path', default=home_dir, type=str, help='Project home directory')
parser.add_argument('--subset', action='store_true', default=False, help='use the subset of datasets')
parser.add_argument('--log_epoch', default=5, type=int, help='print loss and metrix')
parser.add_argument('--draw_similar_matrix', default=10, type=int, help='draw similarity matrix')
parser.add_argument('--pretrain_lr', default=0.0001, type=float, help='pretrain learning rate')
parser.add_argument('--lr', default=0.0001, type=float, help='learning rate')
parser.add_argument('--use_pretrain_epoch_dir', default=None, type=str,
                    help='choose the pretrain checkpoint to finetune')
parser.add_argument('--pretrain_epoch', default=10, type=int, help='pretrain epochs')
parser.add_argument('--finetune_epoch', default=300, type=int, help='finetune epochs')

parser.add_argument('--masking_ratio', default=0.5, type=float, help='masking ratio')
parser.add_argument('--positive_nums', default=3, type=int, help='positive series numbers')
parser.add_argument('--lm', default=3, type=int, help='average masked lenght')

parser.add_argument('--finetune_result_file_name', default="finetune_result.json", type=str,
                    help='finetune result json name')
parser.add_argument('--temperature', type=float, default=0.2, help='temperature')

args, unknown = parser.parse_known_args()


# Convert csv to pt

In [ ]:

def csv_to_pt(patient_files, lengths, is_sepsis, desc):
    
    all_patients = {'samples': [], 'labels': []}
    
    max_time_step = 336
    # print(len(patient_files), len(lengths), len(is_sepsis))
    for idx, (file, length, sepsis) in tqdm.tqdm(enumerate(zip(patient_files, lengths, is_sepsis)), 
                                                      desc=f"{desc}", 
                                                      total=len(patient_files)):
        
        pad_width = ((0, max_time_step - len(file)), (0, 0))
        file = np.pad(file, pad_width=pad_width, mode='constant').astype(np.float32)
        
        if len(file) == max_time_step:
            all_patients['samples'].append(torch.from_numpy(file).unsqueeze(0))
            all_patients['labels'].append(torch.tensor(sepsis, dtype=torch.float32).unsqueeze(0))
        else:
            raise ValueError(f"Length {length} does not match length of patient {idx} with length {len(file)}")
    
    # print('samples: ', type(all_patients['samples']), 'labels: ', type(all_patients['labels']))
    
    all_patients['samples'] = torch.cat(all_patients['samples'], dim=0)
    all_patients['labels'] = torch.cat(all_patients['labels'], dim=0)
    
    return {'samples': all_patients['samples'], 'labels': all_patients['labels']}, lengths, is_sepsis

# all_patients, lengths, is_sepsis = csv_to_pt()


In [ ]:
from sklearn.model_selection import train_test_split

def get_train_val_test_indices(sepsis_file, dset, save_distributions=True):
    
    sepsis = pd.read_csv(os.path.join(project_root(), 'data', 'tl_datasets', f'{sepsis_file}'), header=None)
    
    train_indices, val_indices = train_test_split(sepsis, test_size=0.2, random_state=2024)
    
    train_indices = train_indices.index.values
    val_indices = val_indices.index.values
    
    
    if save_distributions:
        train_dist = sepsis.iloc[train_indices].value_counts()
        val_dist = sepsis.iloc[val_indices].value_counts()
        
        train_dist_percentage = np.round(train_dist / len(sepsis.iloc[train_indices]), 2)
        val_dist_percentage = np.round(val_dist / len(sepsis.iloc[val_indices]), 2)
        
        pd.DataFrame(
            {
                'Train Images': train_dist, 'Validation Images': val_dist,
                'Train Distribution Percentage': train_dist_percentage, 'Validation Distribution Percentage': val_dist_percentage,
            }
        ).to_csv(os.path.join(project_root(), 'results', f'distributions{dset}.csv'), index=False)
        
        # pd.read_csv(os.path.join(project_root(), 'results', 'distributions.csv'))
        
    return train_indices, val_indices


# Datasetup

In [ ]:
def get_pretrain_finetune_datasets():
    
    # Pre-training Indices
    pt_train_indices, pt_val_indices = get_train_val_test_indices(
        sepsis_file='is_sepsis_pretrain_A.txt', save_distributions=True,
        dset='Aa')
    
    # Gathering files, lengths, and sepsis label
    pt_files = pd.read_pickle(os.path.join(project_root(), 'data', 'tl_datasets', 'final_dataset_pretrain_A.pickle'))
    pt_lengths = pd.read_csv(os.path.join(project_root(), 'data', 'tl_datasets', 'lengths_pretrain_A.txt'), 
                             header=None)
    pt_sepsis = pd.read_csv(os.path.join(project_root(), 'data', 'tl_datasets', 'is_sepsis_pretrain_A.txt'),
                            header=None)
    
    # Checking whether the files are in same order or not
    pretrain_files = []
    for pdata, length in tqdm.tqdm(zip(pt_files, pt_lengths.values), desc="Checking Pre-training & Validation Files", 
                                   total=len(pt_files)):
        plength = len(pdata) 
        assert plength == length[0], f"{plength} doesn't match {length}"
        pretrain_files.append(pdata.drop(['PatientID', 'SepsisLabel'], axis=1))
    
    # Getting train and val
    pt_train = [pretrain_files[i] for i in pt_train_indices]
    pt_val = [pretrain_files[i] for i in pt_val_indices]
    
    pt_train_lengths = pt_lengths.iloc[pt_train_indices].values
    pt_val_lengths = pt_lengths.iloc[pt_val_indices].values
    
    pt_train_sepsis = pt_sepsis.iloc[pt_train_indices].values
    pt_val_sepsis = pt_sepsis.iloc[pt_val_indices].values
    
    pt_train, pt_train_lengths, pt_train_sepsis = csv_to_pt(pt_train, pt_train_lengths, pt_train_sepsis, desc='PT Train Set')
    pt_val, pt_val_lengths, pt_val_sepsis = csv_to_pt(pt_val, pt_val_lengths, pt_val_sepsis, desc='PT Validation Set')
    
    # Fine-tuning
    test_indices, finetune_indices = get_train_val_test_indices(
        sepsis_file='is_sepsis_finetune_B.txt', save_distributions=True,
        dset='Bb')
    
    # Gathering files, lengths, and sepsis label
    test_setB = pd.read_pickle(os.path.join(project_root(), 'data', 'tl_datasets', 'final_dataset_finetune_B.pickle'))
    test_setB_lengths = pd.read_csv(os.path.join(project_root(), 'data', 'tl_datasets', 'lengths_finetune_B.txt'), 
                             header=None)
    test_setB_sepsis = pd.read_csv(os.path.join(project_root(), 'data', 'tl_datasets', 'is_sepsis_finetune_B.txt'),
                            header=None)
    
    # Checking whether the files are in same order or not
    test_files = []
    for pdata, length in tqdm.tqdm(zip(test_setB, test_setB_lengths.values), desc="Checking Fine-tuning & Test Files",
                                   total=len(test_setB)):
        plength = len(pdata) 
        assert plength == length[0], f"{plength} doesn't match {length}"
        test_files.append(pdata.drop(['PatientID', 'SepsisLabel'], axis=1))
    
    # Getting finetune and test sets
    finetune = [test_files[i] for i in finetune_indices]
    test = [test_files[i] for i in test_indices]
    
    finetune_lengths = test_setB_lengths.iloc[finetune_indices].values
    test_lengths = test_setB_lengths.iloc[test_indices].values
    
    finetune_sepsis = test_setB_sepsis.iloc[finetune_indices].values
    test_sepsis = test_setB_sepsis.iloc[test_indices].values
    
    finetune, finetune_lengths, finetune_sepsis = csv_to_pt(finetune, finetune_lengths, finetune_sepsis, desc="Fine-tuning Set")
    test, test_lengths, test_sepsis = csv_to_pt(test, test_lengths, test_sepsis, desc="Test Set")
    
    print("Pre-training samples: ", pt_train['samples'].shape, "Validation samples: ", pt_val['samples'].shape)
    print("Fine-tuning samples: ", finetune['samples'].shape, "Test samples: ", test['samples'].shape)
    
    return pt_train, pt_val, finetune, test
    
pt_train, pt_val, finetune, test = get_pretrain_finetune_datasets()


In [ ]:

import math

def geom_noise_mask_single(L, lm, masking_ratio):
    """
    Randomly create a boolean mask of length `L`, consisting of subsequences of average length lm, masking with 0s a `masking_ratio`
    proportion of the sequence L. The length of masking subsequences and intervals follow a geometric distribution.
    Args:
        L: length of mask and sequence to be masked
        lm: average length of masking subsequences (streaks of 0s)
        masking_ratio: proportion of L to be masked
    Returns:
        (L, ) boolean numpy array intended to mask ('drop') with 0s a sequence of length L
    """
    keep_mask = np.ones(L, dtype=bool)
    p_m = 1 / lm  # probability of each masking sequence stopping. parameter of geometric distribution.
    p_u = p_m * masking_ratio / (
            1 - masking_ratio)  # probability of each unmasked sequence stopping. parameter of geometric distribution.
    p = [p_m, p_u]

    # Start in state 0 with masking_ratio probability
    state = int(np.random.rand() > masking_ratio)  # state 0 means masking, 1 means not masking
    for i in range(L):
        keep_mask[i] = state  # here it happens that state and masking value corresponding to state are identical
        if np.random.rand() < p[state]:
            state = 1 - state

    return keep_mask


def noise_mask(X, masking_ratio=0.25, lm=3, distribution='geometric', exclude_feats=None):
    """
    Creates a random boolean mask of the same shape as X, with 0s at places where a feature should be masked.
    Args:
        X: (seq_length, feat_dim) numpy array of features corresponding to a single sample
        masking_ratio: proportion of seq_length to be masked. At each time step, will also be the proportion of
            feat_dim that will be masked on average
        lm: average length of masking subsequences (streaks of 0s). Used only when `distribution` is 'geometric'.
        distribution: whether each mask sequence element is sampled independently at random, or whether
            sampling follows a markov chain (and thus is stateful), resulting in geometric distributions of
            masked squences of a desired mean length `lm`
        exclude_feats: iterable of indices corresponding to features to be excluded from masking (i.e. to remain all 1s)
    Returns:
        boolean numpy array with the same shape as X, with 0s at places where a feature should be masked
    """
    if exclude_feats is not None:
        exclude_feats = set(exclude_feats)

    if distribution == 'geometric':  # stateful (Markov chain)
        mask = geom_noise_mask_single(X.shape[0] * X.shape[1] * X.shape[2], lm, masking_ratio)
        mask = mask.reshape(X.shape[0], X.shape[1], X.shape[2])
        
    elif distribution == 'masked_tail':
        mask = np.ones(X.shape, dtype=bool)
        for m in range(X.shape[0]):  # feature dimension

            keep_mask = np.zeros_like(mask[m, :], dtype=bool)
            n = math.ceil(keep_mask.shape[1] * (1 - masking_ratio))
            keep_mask[:, :n] = True
            mask[m, :] = keep_mask  # time dimension
            
    elif distribution == 'masked_head':
        mask = np.ones(X.shape, dtype=bool)
        for m in range(X.shape[0]):  # feature dimension

            keep_mask = np.zeros_like(mask[m, :], dtype=bool)
            n = math.ceil(keep_mask.shape[1] * masking_ratio)
            keep_mask[:, n:] = True
            mask[m, :] = keep_mask  # time dimension
    else:  # each position is independent Bernoulli with p = 1 - masking_ratio
        mask = np.random.choice(np.array([True, False]), size=X.shape, replace=True,
                                p=(1 - masking_ratio, masking_ratio))

    return torch.tensor(mask)

def data_transform_masked4cl(sample, masking_ratio, lm, positive_nums=None, distribution='geometric'):
    """Masked time series in time dimension"""

    if positive_nums is None:
        positive_nums = math.ceil(1.5 / (1 - masking_ratio))
        
    sample = sample.permute(0, 2, 1)  # (batch_size, channels, time_steps)
    
    # Creating the batch in #positive_nums sets
    sample_repeat = sample.repeat(positive_nums, 1, 1)  # (batch_size*positive_num, channels, time steps)

    mask = noise_mask(sample_repeat, masking_ratio, lm, distribution=distribution)
    x_masked = mask * sample_repeat

    return x_masked.permute(0, 2, 1), mask.permute(0, 2, 1)

# data_masked_m, mask = data_transform_masked4cl(all_patients['samples'][:32], 0.5, 3, positive_nums=1, distribution='geometric')


In [ ]:
from torch.utils.data import Dataset


class Load_Dataset(Dataset):
    
    def __init__(self, dataset, TSlength_aligned, training_mode):
        
        super(Load_Dataset, self).__init__()
        self.training_mode = training_mode
        
        X_train = dataset["samples"]
        y_train = dataset["labels"]
        
        # shuffle
        data = list(zip(X_train, y_train))
        np.random.shuffle(data)
        
        X_train, y_train = zip(*data)
        X_train, y_train = torch.stack(list(X_train), dim=0), torch.stack(list(y_train), dim=0)

        if len(X_train.shape) < 3:
            X_train = X_train.unsqueeze(2)

        # if X_train.shape.index(min(X_train.shape)) != 1:  # make sure the Channels in second dim
        #     X_train = X_train.permute(0, 2, 1)

        """Align the TS length between source and target datasets"""
        # X_train = X_train[:, :1, :int(config.TSlength_aligned)] # take the first 178 samples
        X_train = X_train[:, :, :int(TSlength_aligned)]
        
        if isinstance(X_train, np.ndarray):
            self.x_data = torch.from_numpy(X_train)
            self.y_data = torch.from_numpy(y_train).long()
        else:
            self.x_data = X_train
            self.y_data = y_train

        self.len = X_train.shape[0]

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.len
    

In [ ]:
import math

from torch import nn
import torch
from torch.nn import ModuleList

import torch.nn.functional as F

from models.simmtm.gtn.encoder import Encoder
from simmtm.loss import ContrastiveWeight, AggregationRebuild, AutomaticWeightedLoss


class TFC(nn.Module):
    def __init__(self, configs, args):
        
        super(TFC, self).__init__()
        self.training_mode = 'pre_train'
        
        # Projecting input into deep representations
        self.encoder_list_1 = ModuleList([Encoder(d_model=configs.d_model, d_hidden=configs.d_hidden, q=configs.q,
                                                  v=configs.v, h=configs.h, mask=configs.mask, dropout=configs.dropout,
                                                  device=configs.device) for _ in range(configs.N)])

        self.encoder_list_2 = ModuleList([Encoder(d_model=configs.d_model, d_hidden=configs.d_hidden, q=configs.q,
                                                  v=configs.v, h=configs.h, dropout=configs.dropout,
                                                  device=configs.device) for _ in range(configs.N)])

        self.embedding_channel = torch.nn.Linear(configs.d_channel, configs.d_model)
        self.embedding_input = torch.nn.Linear(configs.d_input, configs.d_model)

        self.gate = torch.nn.Linear(configs.d_model * configs.d_input + configs.d_model * configs.d_channel,
                                    configs.d_output)

        self.pe = configs.pe
        self._d_input = configs.d_input
        self._d_model = configs.d_model

        # MLP Layer - To generate Projector(.); to Obtain series-wise representations
        self.dense = nn.Sequential(
            nn.Linear(192512, 256),  # 240128 = encoder1 out features + encoder2 out features
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Linear(256, 128)
        )

        if self.training_mode == 'pre_train':
            self.awl = AutomaticWeightedLoss(2)
            self.contrastive = ContrastiveWeight(args)
            self.aggregation = AggregationRebuild(args)
            # self.head = nn.Linear(240128, 336)  # Reconstruction, we have 336 time steps
            self.head = nn.Linear(192512, configs.d_input * configs.d_channel)  # Replaced to handle multi-variate
            self.mse = torch.nn.MSELoss()
            
    def forward(self, stage, x_in_t, pre_train=False):

        # x_in_t: (128, 336, 133)
        encoding_1 = self.embedding_channel(x_in_t)  # (128, 336, 512)
        input_to_gather = encoding_1 

        if self.pe:
            pe = torch.ones_like(encoding_1[0])
            position = torch.arange(0, self._d_input).unsqueeze(-1)
            temp = torch.Tensor(range(0, self._d_model, 2))
            temp = temp * -(math.log(10000) / self._d_model)
            temp = torch.exp(temp).unsqueeze(0)
            temp = torch.matmul(position.float(), temp)  # shape:[input, d_model/2]
            pe[:, 0::2] = torch.sin(temp)
            pe[:, 1::2] = torch.cos(temp)

            encoding_1 = encoding_1 + pe  # (128, 336, 512)

        for encoder in self.encoder_list_1:
            # encoding_1: (128, 336, 512)
            encoding_1, score_input = encoder(encoding_1, stage)
            
        encoding_2 = self.embedding_input(x_in_t.transpose(-1, -2))  # encoding_2: (128, 133, 512)
        channel_to_gather = encoding_2  

        for encoder in self.encoder_list_2:
            # encoding_2: (128, 133, 512)
            encoding_2, score_channel = encoder(encoding_2, stage)

        encoding_1 = encoding_1.reshape(encoding_1.shape[0], -1)  # (128, 172032)
        encoding_2 = encoding_2.reshape(encoding_2.shape[0], -1)  # (128, 68096)
        
        encoding_concat = self.gate(torch.cat([encoding_1, encoding_2], dim=-1))  # (128, 2)
        
        # gate: torch.Size([128, 2])
        gate = F.softmax(encoding_concat, dim=-1)  
        encoding = torch.cat([encoding_1 * gate[:, 0:1], encoding_2 * gate[:, 1:2]], dim=-1)  # (128, 240128)
        # print(encoding.shape)
        
        # Projections
        projections = self.dense(encoding)  # (128, 128)

        if pre_train:
            # loss_cl: torch.Size([])
            # similarity_matrix: torch.Size([128, 128])
            # logits: torch.Size([128, 127])
            # positives_mask: torch.Size([128, 128])
            loss_cl, similarity_matrix, logits, positives_mask = self.contrastive(projections)           
            
            # rebuild_weight_matrix: torch.Size([128, 128])
            # agg_x: torch.Size([128, 240128])
            rebuild_weight_matrix, agg_x = self.aggregation(similarity_matrix, encoding)
            
            # pred_x: torch.Size([128, 336])
            pred_x = self.head(agg_x.reshape(agg_x.size(0), -1))
            
            # x_in_t.shape: torch.Size([128, 336, 133])
            # x_in_t.reshape(x_in_t.size(0), -1): torch.Size([128, 44688])
            loss_rb = self.mse(pred_x, x_in_t.reshape(x_in_t.size(0), -1).detach())
            loss = self.awl(loss_cl, loss_rb)

            return loss, loss_cl, loss_rb

        return encoding, encoding_concat
    

In [ ]:

def model_pretrain(model, model_optimizer, model_scheduler, train_loader, configs, args, device):
    total_loss = []
    total_cl_loss = []
    total_rb_loss = []
    
    model.to(device)
    model.train()
    for batch_idx, (data, labels) in tqdm.tqdm(enumerate(train_loader), desc="Pre-training model", total=len(train_loader)):  # data shape: (batch_size, seqs, channels)

        model_optimizer.zero_grad()
        # When masking, data is reshaped to (batch_size, channel, seqs) - Inside the data_transform_masked4cl()
        data_masked_m, mask = data_transform_masked4cl(data, args.masking_ratio, args.lm, args.positive_nums)
        data_masked_om = torch.cat([data, data_masked_m], 0)  # (batch_size, seqs, channels)

        data, labels = data.float().to('cpu'), labels.float().to('cpu')
        data_masked_om = data_masked_om.float().to(device)

        # Produce embeddings of original and masked samples  (data_masked_om = data samples + masked samples)
        # loss, loss_cl, loss_rb = model(data_masked_om, pretrain=True)
        # return loss, loss_cl, loss_rb
        
        loss, loss_cl, loss_rb = model(stage='train', x_in_t=data_masked_om, pre_train=True)
        
        # return loss, loss_cl, loss_rb

        loss.backward()
        model_optimizer.step()

        total_loss.append(loss.item())
        total_cl_loss.append(loss_cl.item())
        total_rb_loss.append(loss_rb.item())

    total_loss = torch.tensor(total_loss).mean()
    total_cl_loss = torch.tensor(total_cl_loss).mean()
    total_rb_loss = torch.tensor(total_rb_loss).mean()

    model_scheduler.step()

    return total_loss, total_cl_loss, total_rb_loss


In [ ]:
pt_dataset = Load_Dataset(pt_train, TSlength_aligned=336, training_mode='pretrain')
train_loader = torch.utils.data.DataLoader(dataset=pt_dataset, batch_size=32, shuffle=True, 
                                           drop_last=True, num_workers=4)  # (32, 336, 40)

val_dataset = Load_Dataset(pt_val, TSlength_aligned=336, training_mode='pretrain')
val_loader = torch.utils.data.DataLoader(dataset=pt_val, batch_size=32, shuffle=True, 
                                           drop_last=True, num_workers=4)

finetune_dataset = Load_Dataset(finetune, TSlength_aligned=336, training_mode='finetune')
finetune_loader = torch.utils.data.DataLoader(finetune_dataset, batch_size=32, shuffle=True, 
                                              drop_last=True, num_workers=4)



# Training

In [ ]:

def get_model_size(model):
    
    def convert_to_gigabytes(input_megabyte):
        gigabyte = 1.0/1024
        convert_gb = gigabyte * input_megabyte
        return convert_gb
    
    param_size = 0
    for param in model.parameters():
        param_size += param.nelement() * param.element_size()
        
    buffer_size = 0
    for buffer in model.buffers():
        buffer_size += buffer.nelement() * buffer.element_size()
    
    size_all_mb = (param_size + buffer_size) / 1024**2
    
    print('model size: {:.3f} GB'.format(convert_to_gigabytes(size_all_mb)))
    
    return convert_to_gigabytes(size_all_mb)


In [ ]:
from models.simmtm.model import target_classifier

from models.simmtm.config import Config

def build_model(args, lr, configs, device='cuda', chkpoint=None):
    
    model = TFC(configs, args).to(device)
    if chkpoint:
        pretrained_dict = chkpoint["model_state_dict"]
        model_dict = model.state_dict()
        model_dict.update(pretrained_dict)
        model.load_state_dict(model_dict)

    classifier = target_classifier(configs).to(device)
    model_optimizer = torch.optim.Adam(model.parameters(), lr=lr, betas=(configs.beta1, configs.beta2), weight_decay=0)
    classifier_optimizer = torch.optim.Adam(classifier.parameters(), lr=lr, 
                                            betas=(configs.beta1, configs.beta2),
                                            weight_decay=0)
    model_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer=model_optimizer, T_max=args.finetune_epoch)

    return model, classifier, model_optimizer, classifier_optimizer, model_scheduler

# model, classifier, model_optimizer, classifier_optimizer, model_scheduler = build_model(args, Config().lr, Config())


In [ ]:

from sklearn.metrics import (roc_auc_score, average_precision_score, accuracy_score, 
                             precision_score, f1_score, recall_score)

def model_finetune(model, val_dl, device, model_optimizer, model_scheduler, classifier=None, classifier_optimizer=None):
    model.train()
    classifier.train()

    total_loss = []
    total_acc = []
    total_auc = []
    total_prc = []

    criterion = nn.CrossEntropyLoss()
    outs = np.array([])
    trgs = np.array([])

    for data, labels in val_dl:
        model_optimizer.zero_grad()
        classifier_optimizer.zero_grad()

        data, labels = data.float().to(device), labels.long().to(device)

        # Produce embeddings
        h, z = model(stage='train', x_in_t=data, pre_train=False)

        # Add supervised classifier: 1) it's unique to finetuning. 2) this classifier will also be used in test
        fea_concat = h

        predictions = classifier(fea_concat)
        fea_concat_flat = fea_concat.reshape(fea_concat.shape[0], -1)
        print(predictions)
        print(labels)
        print(predictions.shape, labels.shape)
        loss = criterion(predictions, labels)

        acc_bs = labels.eq(predictions.detach().argmax(dim=1)).float().mean()
        onehot_label = F.one_hot(labels)
        pred_numpy = predictions.detach().cpu().numpy()

        try:
            auc_bs = roc_auc_score(onehot_label.detach().cpu().numpy(), pred_numpy, average="macro", multi_class="ovr")
        except:
            auc_bs = 0.0

        try:
            prc_bs = average_precision_score(onehot_label.detach().cpu().numpy(), pred_numpy)
        except:
            prc_bs = 0.0

        total_acc.append(acc_bs)

        if auc_bs != 0:
            total_auc.append(auc_bs)
        if prc_bs != 0:
            total_prc.append(prc_bs)
        total_loss.append(loss.item())

        loss.backward()
        model_optimizer.step()
        classifier_optimizer.step()

        pred = predictions.max(1, keepdim=True)[1]
        outs = np.append(outs, pred.cpu().numpy())
        trgs = np.append(trgs, labels.data.cpu().numpy())

    labels_numpy = labels.detach().cpu().numpy()
    pred_numpy = np.argmax(pred_numpy, axis=1)
    F1 = f1_score(labels_numpy, pred_numpy, average='macro', )  # labels=np.unique(ypred))

    total_loss = torch.tensor(total_loss).mean()  # average loss
    total_acc = torch.tensor(total_acc).mean()  # average acc
    total_auc = torch.tensor(total_auc).mean()  # average auc
    total_prc = torch.tensor(total_prc).mean()

    model_scheduler.step(total_loss)

    return total_loss, total_acc, total_auc, total_prc, fea_concat_flat, trgs, F1


In [ ]:

def train(train_loader, val_loader, finetune_loader, device='cuda'):
    
    model = TFC(configs=Config(), args=args)
    params_group = [{'params': model.parameters()}]
    model_optimizer = torch.optim.Adam(params_group, lr=args.pretrain_lr, 
                                       betas=(Config().beta1, Config().beta2),
                                       weight_decay=0)
    
    model_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer=model_optimizer, T_max=args.pretrain_epoch)

    experiment_log_dir = os.path.join(project_root(), 'results', 'simmtm')
    os.makedirs(os.path.join(experiment_log_dir, f"saved_models"), exist_ok=True)
    
    best_performance = None
    seed = 2024
    for epoch in range(Config().pretrain_epoch):
        total_loss, total_cl_loss, total_rb_loss = model_pretrain(model=model, model_optimizer=model_optimizer,
                                                              model_scheduler=model_scheduler, train_loader=train_loader, 
                                                              configs=Config(), args=args, device='cuda')
        
        print(f'Pre-training Epoch: {epoch}\t Train Loss: {total_loss:.4f}\t CL Loss: {total_cl_loss:.4f}\t RB Loss: {total_rb_loss:.4f}\n')
        
        chkpoint = {'seed': seed, 'epoch': epoch, 'train_loss': total_loss, 'model_state_dict': model.state_dict()}
        torch.save(chkpoint, os.path.join(experiment_log_dir, f"saved_models/", f'ckp_ep{epoch}.pt'))
        
        # if epoch % 2 == 0:
        for ep in range(1, Config().finetune_epoch+1):
            print(f"Fine-tuning started...")
            ft_model, ft_classifier, ft_model_optimizer, ft_classifier_optimizer, ft_scheduler = build_model(
                args, args.lr, Config(), device=device, chkpoint=chkpoint)
            
            for ep in range(1, Config().finetune_epoch):
                valid_loss, valid_acc, valid_auc, valid_prc, emb_finetune, label_finetune, F1 = model_finetune(
                    ft_model, finetune_loader, device, ft_model_optimizer, ft_scheduler, classifier=ft_classifier,
                    classifier_optimizer=ft_classifier_optimizer)
        
        
        # # Loading the model
        # temp_model = TFC(configs=Config(), args=args)
        # 
        # pretrained_dict = chkpoint["model_state_dict"]
        # model_dict = temp_model.state_dict()
        # model_dict.update(pretrained_dict)
        # temp_model.load_state_dict(model_dict)

train(train_loader, val_loader, finetune_loader)


In [12]:
from utils.evaluate_helper_methods import load_sepsis_model
from utils.path_utils import project_root
import os

model_path = os.path.join(project_root(), 'results', 'simmtm', 'saved_models', 'finetune_ep16.pt')
model = load_sepsis_model(d_input=336, d_channel=40, d_output=2, model_name=model_path,
                          pre_model="simmtm")


In [24]:
torch.load(model_path)['classifier']

OrderedDict([('logits.weight',
              tensor([[-0.0013, -0.0006, -0.0009,  ...,  0.0029, -0.0038, -0.0021],
                      [ 0.0014,  0.0022, -0.0012,  ..., -0.0052,  0.0039,  0.0008],
                      [ 0.0004, -0.0007,  0.0008,  ..., -0.0010, -0.0008, -0.0038],
                      ...,
                      [ 0.0002, -0.0008, -0.0016,  ...,  0.0002,  0.0033, -0.0005],
                      [-0.0012,  0.0015,  0.0017,  ..., -0.0072,  0.0074,  0.0077],
                      [ 0.0015, -0.0015,  0.0009,  ...,  0.0008,  0.0025,  0.0017]],
                     device='cuda:0')),
             ('logits.bias',
              tensor([-2.5756e-03,  2.3701e-03, -8.7043e-04, -2.2944e-03,  8.7966e-04,
                      -1.0887e-03, -9.9310e-04, -1.3832e-03, -1.2956e-03, -2.0353e-03,
                       5.4989e-05, -4.9039e-04,  8.6841e-04, -7.9175e-04, -3.5971e-03,
                       3.8617e-03, -1.0247e-04, -3.8091e-03, -3.1552e-03, -3.6906e-04,
                    

In [26]:
configs = Config()
classifier = target_classifier(configs=configs)

target_classifier(
  (logits): Linear(in_features=192512, out_features=64, bias=True)
  (logits_simple): Linear(in_features=64, out_features=2, bias=True)
)

In [15]:
import torch

test_data = torch.load(os.path.join(project_root(), 'data', 'test_data', 'simmtm', 'test.pt'))['samples']


In [41]:
test_setB_all_files = os.path.join(project_root(), 'physionet.org', 'files', 'challenge-2019', '1.0.0', 'training',
                         'training_setB')
test_setB_files = os.listdir(test_setB_all_files)
test_setB_files.sort()
test_setB_files.remove('index.html')

test_setB_files = [test_setB_files[i] for i in test_indices]


In [67]:
save_path = os.path.join(project_root(), 'data', 'test_data', 'simmtm', 'psv_files')
for pidx in test_setB_files:
    pdata = pd.read_csv(os.path.join(test_setB_all_files, pidx), sep='|')
    pdata.to_csv(os.path.join(save_path, pidx), sep='|', index=False)


In [4]:
import pandas as pd
import os

from utils.path_utils import project_root
from utils.pretrain_utils.data import get_train_val_test_indices

test_indices, finetune_indices = get_train_val_test_indices(
        sepsis_file='is_sepsis_finetune_B.txt', save_distributions=True,
        dset='Bb')

test_setB_all_files = os.path.join(project_root(), 'physionet.org', 'files', 'challenge-2019', '1.0.0', 'training',
                         'training_setB')
test_setB_files = os.listdir(test_setB_all_files)
test_setB_files.sort()
test_setB_files.remove('index.html')

test_setB_files = [test_setB_files[i] for i in test_indices]

for pidx in test_setB_files:
    break
    
save_path = os.path.join(project_root(), 'data', 'test_data', 'simmtm', 'psv_files')
pd.read_csv(os.path.join(save_path, pidx), sep='|')

,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,EtCO2,BaseExcess,HCO3,...,WBC,Fibrinogen,Platelets,Age,Gender,Unit1,Unit2,HospAdmTime,ICULOS,SepsisLabel
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,23,1,NaN,NaN,-28.85,1,0
1,72.0,100.0,36.6,135.0,108.0,94.0,15.0,NaN,NaN,NaN,...,NaN,NaN,NaN,23,1,NaN,NaN,-28.85,2,0
2,102.0,100.0,NaN,142.0,115.0,86.0,14.0,NaN,NaN,NaN,...,NaN,NaN,NaN,23,1,NaN,NaN,-28.85,3,0
3,78.0,96.0,NaN,133.0,108.0,95.0,16.0,NaN,NaN,NaN,...,13.4,NaN,69.0,23,1,NaN,NaN,-28.85,4,0
4,80.0,97.0,NaN,131.0,111.0,95.0,16.0,NaN,NaN,NaN,...,NaN,NaN,NaN,23,1,NaN,NaN,-28.85,5,0
5,84.0,98.0,36.2,137.0,107.0,94.0,15.0,NaN,NaN,NaN,...,NaN,NaN,NaN,23,1,NaN,NaN,-28.85,6,0
6,78.0,97.0,NaN,135.0,109.0,94.0,16.0,NaN,NaN,NaN,...,NaN,NaN,NaN,23,1,NaN,NaN,-28.85,7,0
7,74.0,97.0,NaN,132.0,109.0,87.0,14.0,NaN,NaN,NaN,...,NaN,NaN,NaN,23,1,NaN,NaN,-28.85,8,0
8,80.0,98.0,NaN,150.0,114.0,90.0,15.0,NaN,NaN,NaN,...,NaN,NaN,NaN,23,1,NaN,NaN,-28.85,9,0
9,78.0,98.0,36.8,148.0,108.0,93.0,14.0,NaN,NaN,NaN,...,NaN,NaN,NaN,23,1,NaN,NaN,-28.85,10,0


In [7]:
import torch
# torch.load(os.path.join(project_root(), 'results', 'simmtm', 'saved_models', 'ckp_ep9.pt'))['model_state_dict']

In [9]:
from models.simmtm.config import Config
from models.simmtm.model import TFC, target_classifier

config = Config()
classifier = target_classifier(config).to('cuda')


In [11]:
[8, 15 (0.89), 16 (1), 17 (1)]